<a href="https://colab.research.google.com/github/madarshb19/Solving-Rubiks-Cube-Using-Genetic-Algorithm/blob/main/GeneticAlgorithmWithCubeRepresentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pycuber
import pycuber as pc
cube2D = pc.Cube()
finalSolution = pc.Cube()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(finalSolution)

         [y][y][y]
         [y][y][y]
         [y][y][y]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
         [w][w][w]
         [w][w][w]
         [w][w][w]



In [ ]:
import numpy as np
from typing import List

GREEN = "G"
ORANGE = "O"
RED = "R"
WHITE = "W"
YELLOW = "Y"
BLUE = "B"

FRONT = "Front"
LEFT = "Left"
BACK = "Back"
RIGHT = "Right"
TOP = "Top"
BOTTOM = "Bottom"

CLOCKWISE = (1, 0)
COUNTERCLOCKWISE = (0, 1)


class Cube:

    def __init__(self):

        self.faces = {
            FRONT: np.full((3, 3), GREEN),
            LEFT: np.full((3, 3), ORANGE),
            RIGHT: np.full((3, 3), RED),
            TOP: np.full((3, 3), WHITE),
            BOTTOM: np.full((3, 3), YELLOW),
            BACK: np.full((3, 3), BLUE),
        }

        self.moves_lookup = {
            "D": self.D, "D'": self.D_prime, "D2": self.D2,
            "E": self.E, "E'": self.E_prime, "E2": self.E2,
            "U": self.U, "U'": self.U_prime, "U2": self.U2,
            "L": self.L, "L'": self.L_prime, "L2": self.L2,
            "R": self.R, "R'": self.R_prime, "R2": self.R2,
            "M": self.M, "M'": self.M_prime, "M2": self.M2,
            "B": self.B, "B'": self.B_prime, "B2": self.B2,
            "F": self.F, "F'": self.F_prime, "F2": self.F2,
            "S": self.S, "S'": self.S_prime, "S2": self.S2,
            "x": self.x_full, "x'": self.x_prime_full, "x2": self.x2_full,
            "y": self.y_full, "y'": self.y_prime_full, "y2": self.y2_full,
            "z": self.z_full, "z'": self.z_prime_full, "z2": self.z2_full,
        }

        self.move_history = []
        self.fitness = 0  # less is better, it means 0 misplaced sticker
    def execute(self, moves: 'List'):
        for m in moves:
            self.moves_lookup[m]()

        # we assume that the first one is the scramble
        self.move_history.append(moves)
        self.__calculate_fitness()

    def __calculate_fitness(self):
        misplaced_stickers = 0

        for k, face in self.faces.items():
            # centers are fixed in a Rubiks cube
            center = face[1, 1]

            for i in range(0, 3):
                for j in range(0, 3):
                    if face[i, j] != center:
                        misplaced_stickers += 1

        self.fitness = misplaced_stickers

    def is_solved(self):
        if self.fitness == 0:
            return True
        return False

    # X Axis movements - D, E and U

    def D(self):
        self.faces[BOTTOM] = np.rot90(self.faces[BOTTOM], axes=CLOCKWISE)
        self.__swap_x((FRONT, 2), (RIGHT, 2), (BACK, 2), (LEFT, 2))

    def D_prime(self):
        self.faces[BOTTOM] = np.rot90(self.faces[BOTTOM], axes=COUNTERCLOCKWISE)
        self.__swap_x((FRONT, 2), (LEFT, 2), (BACK, 2), (RIGHT, 2))

    def D2(self):
        self.D()
        self.D()

    def E(self):
        self.__swap_x((FRONT, 1), (RIGHT, 1), (BACK, 1), (LEFT, 1))

    def E_prime(self):
        self.__swap_x((FRONT, 1), (LEFT, 1), (BACK, 1), (RIGHT, 1))

    def E2(self):
        self.E()
        self.E()

    def U(self):
        self.faces[TOP] = np.rot90(self.faces[TOP], axes=CLOCKWISE)
        self.__swap_x((FRONT, 0), (LEFT, 0), (BACK, 0), (RIGHT, 0))

    def U_prime(self):
        self.faces[TOP] = np.rot90(self.faces[TOP], axes=COUNTERCLOCKWISE)
        self.__swap_x((FRONT, 0), (RIGHT, 0), (BACK, 0), (LEFT, 0))

    def U2(self):
        self.U()
        self.U()

    def __swap_x(self, t1, t2, t3, t4):
        # t1, t2, t3 and t4 are tuples
        # This means: put t1 into t2, t2 into t3, t3 into t4, t4 into t1
        backup = np.array(["", "", ""])
        self.__copy_stickers(backup, self.faces[t4[0]][t4[1]])
        self.__copy_stickers(self.faces[t4[0]][t4[1]], self.faces[t3[0]][t3[1]])
        self.__copy_stickers(self.faces[t3[0]][t3[1]], self.faces[t2[0]][t2[1]])
        self.__copy_stickers(self.faces[t2[0]][t2[1]], self.faces[t1[0]][t1[1]])
        self.__copy_stickers(self.faces[t1[0]][t1[1]], backup)

    def __copy_stickers(self, destination, origin):
        destination[0] = origin[0]
        destination[1] = origin[1]
        destination[2] = origin[2]

    # Y Axis movements - L, R and M

    def L(self):
        self.faces[LEFT] = np.rot90(self.faces[LEFT], axes=CLOCKWISE)
        self.__swap_y((BOTTOM, 0, True), (BACK, 2, True), (TOP, 0, False), (FRONT, 0, False))

    def L_prime(self):
        self.faces[LEFT] = np.rot90(self.faces[LEFT], axes=COUNTERCLOCKWISE)
        self.__swap_y((BOTTOM, 0, False), (FRONT, 0, False), (TOP, 0, True), (BACK, 2, True))

    def L2(self):
        self.L()
        self.L()

    def M(self):
        self.__swap_y((BOTTOM, 1, True), (BACK, 1, True), (TOP, 1, False), (FRONT, 1, False))

    def M_prime(self):
        self.__swap_y((BOTTOM, 1, False), (FRONT, 1, False), (TOP, 1, True), (BACK, 1, True))

    def M2(self):
        self.M()
        self.M()

    def R(self):
        self.faces[RIGHT] = np.rot90(self.faces[RIGHT], axes=CLOCKWISE)
        self.__swap_y((BOTTOM, 2, False), (FRONT, 2, False), (TOP, 2, True), (BACK, 0, True))

    def R_prime(self):
        self.faces[RIGHT] = np.rot90(self.faces[RIGHT], axes=COUNTERCLOCKWISE)
        self.__swap_y((BOTTOM, 2, True), (BACK, 0, True), (TOP, 2, False), (FRONT, 2, False))

    def R2(self):
        self.R()
        self.R()

    def __swap_y(self, t1, t2, t3, t4):
        # t1, t2, t3 and t4 are 3-tuples
        # Index 0 = face
        # Index 1 = column
        # Index 2 = boolean, flip values
        # This means: put t1 into t2, t2 into t3, t3 into t4, t4 into t1
        backup = np.array(["", "", ""])

        if t4[2]:
            self.__copy_stickers(backup, np.flip(self.faces[t4[0]][:, t4[1]]))
        else:
            self.__copy_stickers(backup, self.faces[t4[0]][:, t4[1]])

        if t3[2]:
            self.__copy_stickers(self.faces[t4[0]][:, t4[1]], np.flip(self.faces[t3[0]][:, t3[1]]))
        else:
            self.__copy_stickers(self.faces[t4[0]][:, t4[1]], self.faces[t3[0]][:, t3[1]])

        if t2[2]:
            self.__copy_stickers(self.faces[t3[0]][:, t3[1]], np.flip(self.faces[t2[0]][:, t2[1]]))
        else:
            self.__copy_stickers(self.faces[t3[0]][:, t3[1]], self.faces[t2[0]][:, t2[1]])

        if t1[2]:
            self.__copy_stickers(self.faces[t2[0]][:, t2[1]], np.flip(self.faces[t1[0]][:, t1[1]]))
        else:
            self.__copy_stickers(self.faces[t2[0]][:, t2[1]], self.faces[t1[0]][:, t1[1]])

        self.__copy_stickers(self.faces[t1[0]][:, t1[1]], backup)

    def B(self):
        self.faces[BACK] = np.rot90(self.faces[BACK], axes=CLOCKWISE)
        self.__swap_z((BOTTOM, 2, True), (RIGHT, 2, False), (TOP, 0, True), (LEFT, 0, False))

    def B_prime(self):
        self.faces[BACK] = np.rot90(self.faces[BACK], axes=COUNTERCLOCKWISE)
        self.__swap_z((BOTTOM, 2, False), (LEFT, 0, True), (TOP, 0, False), (RIGHT, 2, True))

    def B2(self):
        self.B()
        self.B()

    def F(self):
        self.faces[FRONT] = np.rot90(self.faces[FRONT], axes=CLOCKWISE)
        self.__swap_z((BOTTOM, 0, False), (LEFT, 2, True), (TOP, 2, False), (RIGHT, 0, True))

    def F_prime(self):
        self.faces[FRONT] = np.rot90(self.faces[FRONT], axes=COUNTERCLOCKWISE)
        self.__swap_z((BOTTOM, 0, True), (RIGHT, 0, False), (TOP, 2, True), (LEFT, 2, False))

    def F2(self):
        self.F()
        self.F()

    def S(self):
        self.__swap_z((BOTTOM, 1, False), (LEFT, 1, True), (TOP, 1, False), (RIGHT, 1, True))

    def S_prime(self):
        self.__swap_z((BOTTOM, 1, True), (RIGHT, 1, False), (TOP, 1, True), (LEFT, 1, False))

    def S2(self):
        self.S()
        self.S()

    def __swap_z(self, t1, t2, t3, t4):

        backup = np.array(["", "", ""])

        if t4[2]:
            self.__copy_stickers(backup, np.flip(self.faces[t4[0]][:, t4[1]]))
        else:
            self.__copy_stickers(backup, self.faces[t4[0]][:, t4[1]])

        if t3[2]:
            self.__copy_stickers(self.faces[t4[0]][:, t4[1]], np.flip(self.faces[t3[0]][t3[1]]))
        else:
            self.__copy_stickers(self.faces[t4[0]][:, t4[1]], self.faces[t3[0]][t3[1]])

        if t2[2]:
            self.__copy_stickers(self.faces[t3[0]][t3[1]], np.flip(self.faces[t2[0]][:, t2[1]]))
        else:
            self.__copy_stickers(self.faces[t3[0]][t3[1]], self.faces[t2[0]][:, t2[1]])

        if t1[2]:
            self.__copy_stickers(self.faces[t2[0]][:, t2[1]], np.flip(self.faces[t1[0]][t1[1]]))
        else:
            self.__copy_stickers(self.faces[t2[0]][:, t2[1]], self.faces[t1[0]][t1[1]])

        self.__copy_stickers(self.faces[t1[0]][t1[1]], backup)

    # ------------------------------------------------------------------------------------
    # Full rotations
    # ------------------------------------------------------------------------------------
    def x_full(self):
        self.L_prime()
        self.M_prime()
        self.R()

    def x_prime_full(self):
        self.L()
        self.M()
        self.R_prime()

    def x2_full(self):
        self.x_full()
        self.x_full()

    def y_full(self):
        self.U()
        self.E_prime()
        self.D_prime()

    def y_prime_full(self):
        self.U_prime()
        self.E()
        self.D()

    def y2_full(self):
        self.y_full()
        self.y_full()

    def z_full(self):
        self.F()
        self.S()
        self.B_prime()

    def z_prime_full(self):
        self.F_prime()
        self.S_prime()
        self.B()

    def z2_full(self):
        self.z_full()
        self.z_full()
    # Util
    
    def get_face_as_str(self, face):
        m = self.faces[face]
        return f"{m[0, 0]} {m[0, 1]} {m[0, 2]} - {m[1, 0]} {m[1, 1]} {m[1, 2]} - {m[2, 0]} {m[2, 1]} {m[2, 2]}"

    def get_scramble(self):
        return self.move_history[0]

    def get_scramble_str(self):
        return " ".join(self.get_scramble())

    def get_algorithm(self):
        flat_list = [item for sublist in self.move_history[1:] for item in sublist] #0 represents scramble which we dont want to include
        return flat_list

    def get_algorithm_str(self):
        return " ".join(self.get_algorithm())

    def __str__(self):
        state = [f"Scramble: {self.get_scramble_str()}", f"Algorithm: {self.get_algorithm_str()}"]

        for k, v in sorted(self.faces.items()):
            state.append(f"{'{:<8}'.format(k + ':')}  {v[0, 0]} {v[0, 1]} {v[0, 2]}")
            state.append(f"{'{:<8}'.format('')}  {v[1, 0]} {v[1, 1]} {v[1, 2]}"),
            state.append(f"{'{:<8}'.format('')}  {v[2, 0]} {v[2, 1]} {v[2, 2]}")

        return "\n".join(state)

In [ ]:
import random as rnd
import cProfile
import time
import operator
SINGLE_MOVES = ["U", "U'", "U2", "D", "D'", "D2",
                "R", "R'", "R2", "L", "L'", "L2",
                "F", "F'", "F2", "B", "B'", "B2"]

FULL_ROTATIONS = ["x", "x'", "x2", "y", "y'", "y2"]

ORIENTATIONS = ["z", "z'", "z2"]

PERMUTATIONS = [
    # permutes two edges: U face, bottom edge and right edge
    "F' L' B' R' U' R U' B L F R U R' U".split(" "),
    # permutes two edges: U face, bottom edge and left edge
    "F R B L U L' U B' R' F' L' U' L U'".split(" "),
    # permutes two corners: U face, bottom left and bottom right
    "U2 B U2 B' R2 F R' F' U2 F' U2 F R'".split(" "),
    # permutes three corners: U face, bottom left and top left
    "U2 R U2 R' F2 L F' L' U2 L' U2 L F'".split(" "),
    # permutes three centers: F face, top, right, bottom
    "U' B2 D2 L' F2 D2 B2 R' U'".split(" "),
    # permutes three centers: F face, top, right, left
    "U B2 D2 R F2 D2 B2 L U".split(" "),
    # U face: bottom edge <-> right edge, bottom right corner <-> top right corner
    "D' R' D R2 U' R B2 L U' L' B2 U R2".split(" "),
    # U face: bottom edge <-> right edge, bottom right corner <-> left right corner
    "D L D' L2 U L' B2 R' U R B2 U' L2".split(" "),
    # U face: top edge <-> bottom edge, bottom left corner <-> top right corner
    "R' U L' U2 R U' L R' U L' U2 R U' L U'".split(" "),
    # U face: top edge <-> bottom edge, bottom right corner <-> top left corner
    "L U' R U2 L' U R' L U' R U2 L' U R' U".split(" "),
    # permutes three corners: U face, bottom right, bottom left and top left
    "F' U B U' F U B' U'".split(" "),
    # permutes three corners: U face, bottom left, bottom right and top right
    "F U' B' U F' U' B U".split(" "),
    # permutes three edges: F face bottom, F face top, B face top
    "L' U2 L R' F2 R".split(" "),
    # permutes three edges: F face top, B face top, B face bottom
    "R' U2 R L' B2 L".split(" "),
    # H permutation: U Face, swaps the edges horizontally and vertically
    "M2 U M2 U2 M2 U M2".split(" ")
]


class Solver:

    def __init__(self, population_size, max_generations, max_resets, elitism_num):
        self.population_size = population_size
        self.max_generations = max_generations
        self.max_resets = max_resets
        self.elitism_num = elitism_num

    def solve(self, scramble, verbose=False):
        start_time = time.time()

        if verbose:
            print("Starting...")

        for r in range(0, self.max_resets):
            # initialize population
            cubes = []
            for i in range(0, self.population_size):
                cube = Cube()
                cube.execute(scramble)
                # randomize it
                # cube.execute(self.__rnd_rotation())
                cube.execute(self.__rnd_single_move())
                cube.execute(self.__rnd_single_move())
                cubes.append(cube)

            # evolve population
            for g in range(0, self.max_generations):
                # sort by fitness
                cubes.sort(key=operator.attrgetter('fitness'))

                if verbose and g % 20 == 0 and g != 0:
                    print(f"World: {r + 1} - Generation: {g}")
                    print(f"Best solution so far")
                    print(f"{cubes[0].get_algorithm_str()}")
                    print(cube2D(f"{cubes[i].get_algorithm_str()}" + " " + f"{cubes[0].get_algorithm_str()}"))
                    print("")

                # the goal is to minimize the fitness function
                # 0 means that the cube is solved
                for i in range(0, len(cubes)):
                    if cubes[i].fitness == 0:
                        print("Solution found")
                        print(f"World: {r + 1} - Generation: {g + 1}")
                        print(f"Scramble: {cubes[i].get_scramble_str()}")
                        print(f"Solution")
                        print(finalSolution)
                        print(f"{cubes[i].get_algorithm_str()}")
                        print(f"Moves: {len(cubes[i].get_algorithm())}")
                        print(f"{time.time() - start_time} seconds")
                        print("")
                        return

                    # elitism: the best performers move to the next generation without changes
                    if i > self.elitism_num:
                        # copy a random top performer cube
                        self.__copy(cubes[i], cubes[rnd.randint(0, self.elitism_num)])
                        evolution_type = rnd.randint(0, 5)

                        if evolution_type == 0:
                            cubes[i].execute(self.__rnd_permutation())
                        elif evolution_type == 1:
                            cubes[i].execute(self.__rnd_permutation())
                            cubes[i].execute(self.__rnd_permutation())
                        elif evolution_type == 2:
                            cubes[i].execute(self.__rnd_full_rotation())
                            cubes[i].execute(self.__rnd_permutation())
                        elif evolution_type == 3:
                            cubes[i].execute(self.__rnd_orientation())
                            cubes[i].execute(self.__rnd_permutation())
                        elif evolution_type == 4:
                            cubes[i].execute(self.__rnd_full_rotation())
                            cubes[i].execute(self.__rnd_orientation())
                            cubes[i].execute(self.__rnd_permutation())
                        elif evolution_type == 5:
                            cubes[i].execute(self.__rnd_orientation())
                            cubes[i].execute(self.__rnd_full_rotation())
                            cubes[i].execute(self.__rnd_permutation())

            if verbose:
                print(f"Resetting the world")

        # if a solution was found we returned
        print("")
        print(f"Solution not found")
        print(f"{time.time() - start_time} seconds")

    # copy.deepcopy was very slow for the whole cube because it tries to figure out what changed
    def __copy(self, cube_to, cube_from):
        for f in cube_from.faces:
            for i in range(0, 3):
                for j in range(0, 3):
                    cube_to.faces[f][i, j] = cube_from.faces[f][i, j]

        cube_to.move_history = [item for item in cube_from.move_history]
        cube_to.fitness = cube_from.fitness

    def __rnd_single_move(self):
        r = rnd.randint(0, len(SINGLE_MOVES) - 1)
        return [SINGLE_MOVES[r]]

    def __rnd_permutation(self):
        r = rnd.randint(0, len(PERMUTATIONS) - 1)
        return PERMUTATIONS[r]

    def __rnd_full_rotation(self):
        r = rnd.randint(0, len(FULL_ROTATIONS) - 1)
        return [FULL_ROTATIONS[r]]

    def __rnd_orientation(self):
        r = rnd.randint(0, len(ORIENTATIONS) - 1)
        return [ORIENTATIONS[r]]


def main():
    # p = cProfile.Profile()   It is used to find the time required for execution of the program.It is implicitly taken in Google Colab
    # p.enable()
    scramble_str = "R' U' L2 B2 U2 F L2 B' L' B D R B F2 L F R' B2 F' L B' D B2 R2 D' U B2 F' D R2"
    scramble = scramble_str.split(" ")

    population_size = 500
    max_generations = 300
    max_resets = 10
    elitism_num = 100

    print(finalSolution)

    solver = Solver(population_size, max_generations, max_resets, elitism_num)
    solver.solve(scramble, verbose=True)

    # # Disable profiling
    # p.disable()
    # # Print the stats
    # p.print_stats()
    # # Dump the stats to a file
    # p.dump_stats("profile.txt")


if __name__ == '__main__':
    main()

         [y][y][y]
         [y][y][y]
         [y][y][y]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
         [w][w][w]
         [w][w][w]
         [w][w][w]

Starting...
World: 1 - Generation: 20
Best solution so far
L2 F' z2 y' U B2 D2 R F2 D2 B2 L U D' R' D R2 U' R B2 L U' L' B2 U R2 y2 z2 U2 R U2 R' F2 L F' L' U2 L' U2 L F' L' U2 L R' F2 R R' U2 R L' B2 L z x D L D' L2 U L' B2 R' U R B2 U' L2 M2 U M2 U2 M2 U M2 L' U2 L R' F2 R y2 z2 F R B L U L' U B' R' F' L' U' L U' L U' R U2 L' U R' L U' R U2 L' U R' U R' U2 R L' B2 L z' x' M2 U M2 U2 M2 U M2 z y2 F' L' B' R' U' R U' B L F R U R' U z' x' F U' B' U F' U' B U U2 B U2 B' R2 F R' F' U2 F' U2 F R' x' M2 U M2 U2 M2 U M2
         [r][b][y]
         [g][b][y]
         [r][b][g]
[w][o][w][g][y][y][r][g][b][r][r][b]
[w][y][w][r][o][w][b][w][g][r][r][o]
[o][o][y][o][o][o][y][w][g][o][y][w]
         [g][b][b]
         [y][g][g]
         [b][r][w]


World: 1 - Generation: 40
B

In [ ]:
print(finalSolution)

         [y][y][y]
         [y][y][y]
         [y][y][y]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
[r][r][r][g][g][g][o][o][o][b][b][b]
         [w][w][w]
         [w][w][w]
         [w][w][w]



In [ ]:
!pip install pycuber

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pycuber as pc

In [ ]:
new_cube = pc.Cube()
new_cube("R' U' L2 B2 U2 F L2 B' L' B D R B F2 L F R' B2 F' L B' D B2 R2 D' U B2 F' D R2 L L z' F' U B U' F U B' U' z2 y D L D' L2 U L' B2 R' U R B2 U' L2 z' L' U2 L R' F2 R F U' B' U F' U' B U U' B2 D2 L' F2 D2 B2 R' U' F R B L U L' U B' R' F' L' U' L U' x U' B2 D2 L' F2 D2 B2 R' U' y2 z' U' B2 D2 L' F2 D2 B2 R' U' z' L U' R U2 L' U R' L U' R U2 L' U R' U z' M2 U M2 U2 M2 U M2 y' z' U B2 D2 R F2 D2 B2 L U D' R' D R2 U' R B2 L U' L' B2 U R2 D' R' D R2 U' R B2 L U' L' B2 U R2 z2 x F' L' B' R' U' R U' B L F R U R' U z' y F R B L U L' U B' R' F' L' U' L U' z y U B2 D2 R F2 D2 B2 L U U B2 D2 R F2 D2 B2 L U x M2 U M2 U2 M2 U M2 L' U2 L R' F2 R D' R' D R2 U' R B2 L U' L' B2 U R2 D' R' D R2 U' R B2 L U' L' B2 U R2 F' L' B' R' U' R U' B L F R U R' U F' L' B' R' U' R U' B L F R U R' U x M2 U M2 U2 M2 U M2 D L D' L2 U L' B2 R' U R B2 U' L2 D' R' D R2 U' R B2 L U' L' B2 U R2")

In [ ]:
new_cube = new_cube("R' U' L2 B2 U2 F L2 B' L' B D R B F2 L F R' B2 F' L B' D B2 R2 D' U B2 F' D R2")
new_cube

In [ ]:
new_cube = new_cube("L2 F y z F' U B U' F U B' U' z2 x U2 R U2 R' F2 L F' L' U2 L' U2 L F' F R B L U L' U B' R' F' L' U' L U' L U' R U2 L' U R' L U' R U2 L' U R' U x z D L D' L2 U L' B2 R' U R B2 U' L2 z y2 U' B2 D2 L' F2 D2 B2 R' U' F' L' B' R' U' R U' B L F R U R' U F' L' B' R' U' R U' B L F R U R' U y U2 R U2 R' F2 L F' L' U2 L' U2 L F' R' U L' U2 R U' L R' U L' U2 R U' L U' U B2 D2 R F2 D2 B2 L U F R B L U L' U B' R' F' L' U' L U' y2 z' L' U2 L R' F2 R U' B2 D2 L' F2 D2 B2 R' U' y' z' U' B2 D2 L' F2 D2 B2 R' U' z' L' U2 L R' F2 R z2 F' L' B' R' U' R U' B L F R U R' U U B2 D2 R F2 D2 B2 L U U B2 D2 R F2 D2 B2 L U U B2 D2 R F2 D2 B2 L U z2 y' U' B2 D2 L' F2 D2 B2 R' U' z U' B2 D2 L' F2 D2 B2 R' U' z U B2 D2 R F2 D2 B2 L U x M2 U M2 U2 M2 U M2 R' U2 R L' B2 L F R B L U L' U B' R' F' L' U' L U' F R B L U L' U B' R' F' L' U' L U' z2 x' M2 U M2 U2 M2 U M2 D L D' L2 U L' B2 R' U R B2 U' L2 D' R' D R2 U' R B2 L U' L' B2 U R2")
new_cube